# SARIMAX

In [10]:
import warnings
warnings.simplefilter('ignore')

In [1]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
import pmdarima as pm
from pmdarima import model_selection

In [2]:
# load the data
# YYYY-MM-DD
df = pd.read_csv('../../_data/nyc_data.csv', index_col=0, parse_dates=True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [4]:
# Rename variable
df = df.rename(columns={'Demand': 'y'})
df.head()

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [5]:
# Extract regressors
X = df.iloc[:, 1:]
X.head()

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,
2015-01-01,0,0,0,3.68,41.305
2015-01-02,0,0,0,4.73,131.574
2015-01-03,0,0,0,7.23,162.700
2015-01-04,0,0,0,10.96,160.281
2015-01-05,0,0,0,6.92,51.077


## Stationarity

In [6]:
# Test
from statsmodels.tsa.stattools import adfuller

pvalue = adfuller(x=df.y)[1]

# condition to read test
if pvalue < 0.05:
  print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
  print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is not stationary. The p-value is 0.3767770707729089


In [7]:
# differencing
# df.y.diff().dropna()

# Test
pvalue = adfuller(x=df.y.diff().dropna())[1]

# condition to read test
if pvalue < 0.05:
  print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
  print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is stationary. The p-value is 3.355773945628143e-22


## SARIMAX model

In [8]:
# Model
# hourly: 24, daily: 7, weekly:52, monthly:12, quarterly:4
model = pm.ARIMA(order=(1, 1, 1),
                 seasonal_order=(1, 1, 1, 7),
                 X=X,
                 suppress_warning=True,
                 force_stationarity=False)

In [11]:
# Cross-validation
cv = model_selection.RollingForecastCV(h=31,
                                       step=16,
                                       initial=df.shape[0] - 180)

cv_score = model_selection.cross_val_score(model,
                                           y=df.y,
                                           scoring='mean_squared_error',
                                           cv=cv,
                                           verbose=2,
                                           error_score=1000000000000000)

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [13]:
# CV performance
error = np.sqrt(np.average(cv_score))
error

59.9552854187642

## Parameter Tuning

In [14]:
# Grid
param_grid = {'p': [0, 1],
              'd': [0, 1],
              'q': [0, 1],
              'P': [0, 1],
              'D': [0, 1],
              'Q': [0, 1]}

grid = ParameterGrid(param_grid)
len(list(grid))

64

In [ ]:
# Parameter tuning
rmse = []
i = 1
# Parameter loop
for params in grid:
  print(f"\nINFO: {i} / {len(list(grid))}")
  # model
  model = pm.ARIMA(order=(params['p'], params['d'], params['q']),
                  seasonal_order=(params['P'], params['D'], params['Q'], 7),
                  X=X,
                  suppress_warning=True,
                  force_stationarity=False)
  # CV
  cv = model_selection.RollingForecastCV(h=31,
                                         step=16,
                                         initial=df.shape[0] - 180)
  cv_score = model_selection.cross_val_score(model,
                                             y=df.y,
                                             scoring='mean_squared_error',
                                             cv=cv,
                                             verbose=2,
                                             error_score=1000000000000000)

  # Error
  error = np.sqrt(np.average(cv_score))
  rmse.append(error)

  i += 1

In [17]:
# Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,0,0,0,0,0,0,103.922477
1,0,0,0,0,0,1,104.210172
2,0,0,0,0,1,0,102.742861
3,0,0,0,0,1,1,85.968240
4,0,0,0,1,0,0,109.196397
...,...,...,...,...,...,...,...
59,1,1,1,0,1,1,61.599702
60,1,1,1,1,0,0,69.041370
61,1,1,1,1,0,1,60.409042
62,1,1,1,1,1,0,62.280574


In [18]:
# export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("./out/best_params_sarimax.csv")